# Arbitrage Free Surface Demonstration

This notebook demonstrates the arbitrage-free volatility surface construction pipeline in a signle underlying.

All logic is imported from `core/` for visualization only. This notebook does not define or modify any core functionality.

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scripts.run_surface_fit import run_surface_fit
from core.surface_pipeline import build_surface_single_expiry
from core.viz.utils import get_spot

In [ ]:
df = pd.read_parquet("../data/processed/options_cleaned_SPY.parquet")
df = df[df["date"] == df["date"].iloc[0]]

close_series = pd.read_csv("../data/processed/realized_vol_SPY.csv", index_col=0, parse_dates=True)["close"]

In [ ]:
df_exp = df[df["expiration"] == df["expiration"].iloc[0]]

plt.figure(figsize=(6, 4))
plt.scatter(
    np.log(df_exp["strike"] / df_exp["spot"]),
    df_exp["impliedVol"],
    label="Raw market IV",
    alpha=0.7
)
plt.xlabel("log-moneyness")
plt.ylabel("Implied Volatility")
plt.title("Raw Implied Volatility Smile")
plt.legend()
plt.show()

In [ ]:
S = get_spot(close_series, df_exp["date"].iloc[0])
T = df_exp["T"].iloc[0]

surface = build_surface_single_expiry(df_exp, S, r=0.01, q=0.0)

plt.figure(figsize=(6, 4))
plt.scatter(
    np.log(df_exp["strike"] / S),
    df_exp["impliedVol"],
    label="Raw market IV",
    alpha=0.5
)
plt.plot(
    np.log(surface["strike"] / S),
    surface["iv_arbitrage_free"],
    label="Arbitrage-free IV",
    linewidth=2
)
plt.xlabel("log-moneyness")
plt.ylabel("Implied Volatility")
plt.title("Single-Expiry Arbitrage-Free Smile")
plt.legend()
plt.show()

In [ ]:
fits = run_surface_fit(df, close_series)

for f in fits:
    print(f["expiry"], "-> SABR:", f["sabr"] is not None)

## Takeaways

- Raw option quotes often violate static arbitrage constraints
- Enforcing convexity and monotonicity produces smooth, stable smiles
- Calendar arbitrage is enforced separately across maturities on total variance
- Parametric models (SVI, SABR) are fit only afer all invariants are satisfied